# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [427]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm
import re

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)



if response.status_code == 200:
    print("All good!")
    print("==============")
    print("\n")
    info = response.json()
    df = pd.DataFrame(info["nobelPrizes"])
    df= df.dropna(subset=['laureates'])
    
df

All good!




,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN
...,...,...,...,...,...,...,...,...,...
665,2023,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",{'en': 'The Sveriges Riksbank Prize in Economi...,2023-10-09,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN
666,2023,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",2023-10-05,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN
667,2023,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",2023-10-06,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN
668,2023,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",2023-10-03,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN


In [489]:
df.laureates[2][1]

{'id': '463',
 'knownName': {'en': 'Frédéric Passy'},
 'fullName': {'en': 'Frédéric Passy'},
 'portion': '1/2',
 'sortOrder': '2',
 'motivation': {'en': 'for his lifelong work for international peace conferences, diplomacy and arbitration',
  'no': 'for sin  livslange innsats for internasjonale fredskonferanser, diplomati og voldgift'},
 'links': [{'rel': 'laureate',
   'href': 'https://api.nobelprize.org/2/laureate/463',
   'action': 'GET',
   'types': 'application/json'}]}

In [479]:
for x in df.laureates:
    if int(x[0]["id"]) == 462:
        print(x)

[{'id': '462', 'knownName': {'en': 'Henry Dunant'}, 'fullName': {'en': 'Jean Henry Dunant'}, 'portion': '1/2', 'sortOrder': '1', 'motivation': {'en': 'for his humanitarian efforts to help wounded soldiers and create international understanding', 'no': 'for sin humanitære innsats for å hjelpe sårede soldater og skape internasjonal forståelse'}, 'links': [{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laureate/462', 'action': 'GET', 'types': 'application/json'}]}, {'id': '463', 'knownName': {'en': 'Frédéric Passy'}, 'fullName': {'en': 'Frédéric Passy'}, 'portion': '1/2', 'sortOrder': '2', 'motivation': {'en': 'for his lifelong work for international peace conferences, diplomacy and arbitration', 'no': 'for sin  livslange innsats for internasjonale fredskonferanser, diplomati og voldgift'}, 'links': [{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laureate/463', 'action': 'GET', 'types': 'application/json'}]}]


# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [407]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    info = response.json()
    df = pd.DataFrame(info["nobelPrizes"])


df.drop(["categoryFullName","awardYear","links"], axis=1, inplace=True)
df["dateAwarded"] = pd.to_datetime(df["dateAwarded"])
df= df.dropna(subset=['laureates'])
df["category"] = df["category"].map(lambda cat: cat["en"])
df["topMotivation"] = df["laureates"].map(lambda laur: laur[0]["motivation"]["en"])
df["laureates"] = df["laureates"].map(lambda laur: laur[0]["id"])



In [409]:
df

,category,dateAwarded,prizeAmount,prizeAmountAdjusted,laureates,topMotivation
0,Chemistry,1901-11-12,150782,10531894,160,in recognition of the extraordinary services h...
1,Literature,1901-11-14,150782,10531894,569,in special recognition of his poetic compositi...
2,Peace,1901-12-10,150782,10531894,462,for his humanitarian efforts to help wounded s...
3,Physics,1901-11-12,150782,10531894,1,in recognition of the extraordinary services h...
4,Physiology or Medicine,1901-10-30,150782,10531894,293,"for his work on serum therapy, especially its ..."
...,...,...,...,...,...,...
665,Economic Sciences,2023-10-09,11000000,11000000,1034,for having advanced our understanding of women...
666,Literature,2023-10-05,11000000,11000000,1032,for his innovative plays and prose which give ...
667,Peace,2023-10-06,11000000,11000000,1033,for her fight against the oppression of women ...
668,Physics,2023-10-03,11000000,11000000,1026,for experimental methods that generate attosec...


### Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [149]:
import time
from tqdm import tqdm




ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here


def get_gender(laureate):

    # Person
    # Your code here

    # Organizrtion
    # Your code here

def get_birthdate(laureate):

    birth_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    return birth_date

def get_age(laureate):

    birth_date = "Unknown"
    award_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    # Award date
    # Your code here

def get_city(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_country(laureate):

    # Person
    # Your code here

    # Origanization
    # Your code here


def get_continent(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_latitude(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_longitude(laureate):

    # Person
    # Your code here

    # Organization
    # Yuor code here


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        # Your code here

laureates_df = pd.DataFrame(laureates_dict)

laureates_df



IndentationError: expected an indented block after function definition on line 10 (122200105.py, line 19)

In [185]:
ids = [int(item["id"]) for l in df['laureates'].values for item in l]
unique_ids = set(ids)

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()
        print(laureate)

        # Your code here

TypeError: 'float' object is not iterable

# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here

,ID
Country,
USA,296
United Kingdom,91
Germany,84
France,63
Russia,30
...,...
Greece,1
Ghana,1
Faroe Islands (Denmark),1
